This notebook is used to determine which strategy is the best for any particular asset.

In [7]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio.csv")

In [9]:
import yfinance as yf

start_date = "2024-03-01"
end_date = "2025-03-01" # Last day is exclusive

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
        "strategy": None,
        "return": float("-inf"),
    }

In [10]:
# Commission fee based on Webull, which is known for low commission fees:
# https://www.webull.com.sg/pricing
# Regular and Extended Hours (04:00 - 20:00 EST)
# 0.025%*Total Trade Amount (Min. USD 0.50)

def commission(order_size, price):
    return max(0.5, abs(order_size) * price * 0.00025)

## Testing with one asset and one strategy

In [ ]:
from backtesting import Backtest

from strategies.scalping import Scalping

data = portfolio["CZR"]["data"]
data.index = data.index.values.astype("datetime64[D]")
bt = Backtest(
    data, Scalping, cash=1000000, commission=commission
)
stats = bt.run()
bt.plot()

return_pct = stats["Return [%]"]
if return_pct >= portfolio["CZR"]["return"]:
    portfolio["CZR"]["strategy"] = bt._strategy.__name__
    portfolio["CZR"]["return"] = return_pct

In [ ]:
stats

## All assets and all strategies

In [11]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.macd_bollinger_bands_mean_reversion import (
    MACDBollingerBandsMeanReversion,
)
from strategies.michael_harris_price_action import MichaelHarrisPriceAction
from strategies.rsi_divergence import RSIDivergence
from strategies.scalping import Scalping
from strategies.volume_spike_reversal import VolumeSpikeReversal
from strategies.bollinger_bands_breakout import BollingerBandsBreakout

strategies = [
    BollingerBandsBreakout,
    LarryWilliamsPriceAction,
    MACDBollingerBandsMeanReversion,
    MichaelHarrisPriceAction,
    RSIDivergence,
    Scalping,
    VolumeSpikeReversal,
]


cryptocurrencies = ["AVAX-USD", "SOL-USD", "LINK-USD"]

for asset in portfolio:
    for strategy in strategies:
        data = portfolio[asset]["data"]
        data.index = data.index.values.astype("datetime64[D]")
        cash = 1000000 * portfolio[asset]["weight"]
        if asset in cryptocurrencies:
            # https://www.webullpay.com/
            # No Direct Fees: Webull Pay does not charge separate fees for trading cryptocurrencies.
            # Instead, a 1% spread (100 basis points) is included in the buying and selling prices
            # of the crypto assets. This means the cost is integrated into the price you pay or
            # receive when trading.
            bt = Backtest(data, strategy, cash=cash, spread=0.01)
        else:
            bt = Backtest(data, strategy, cash=cash, commission=commission)
        stats = bt.run()
        return_pct = stats["Return [%]"]
        if return_pct >= portfolio[asset]["return"]:
            portfolio[asset]["strategy"] = bt._strategy.__name__
            portfolio[asset]["return"] = return_pct

# NOTE: Many instances of broker cancelling the relative-sized order due to insufficient margin.

Backtest.run:   0%|          | 0/249 [00:00<?, ?it/s]/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/backtesting.py:955: UserWarning: time=29: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/backtesting.py:955: UserWarning: time=30: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/backtesting.py:955: UserWarning: time=31: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/backtesting.py:955: UserWarning: time=32: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/backtesting.py:

In [12]:
results = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["weight"],
            portfolio[asset]["strategy"],
            portfolio[asset]["return"]
        ]
        for asset in portfolio
    ],
    columns=["asset", "weight", "strategy", "return"]
)

results.to_csv("../data/processed/asset_strategies.csv", index=False)